<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/5_Aprendizaje_supervisado/5_Taller_Ensamble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller Ensamble**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

-Juan Galindo
-

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Ensamble_apellidos.ipynb”.

**Plazo de entrega**

13 de noviembre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

## Caso de Consultoría: Ames Real Estate Solutions

**Contexto:** Eres un consultor de datos contratado por **Ames Real Estate Solutions**. Tu misión es construir un *pipeline* de Machine Learning robusto para predecir el precio de venta (`SalePrice`) de las viviendas en la ciudad de Ames, Iowa.

**Objetivos:**
1.  Construir un *pipeline* de preprocesamiento profesional usando `ColumnTransformer` y `OneHotEncoder`.
2.  Entrenar y comparar el desempeño en prueba de 3 modelos de regresión: árbol de decisión, random forest y gradient boosting.
3.  Evaluar los modelos usando **R-cuadrado (R²)** y **RMSE** (Raíz del Error Cuadrático Medio).
4.  Optimizar uno de los modelos, el de Random Forest, usando `GridSearchCV` para mejorar su rendimiento en prueba.

### Paso 1: Configuración Inicial

In [1]:
# Importa las librerías necesarias
### TU CÓDIGO AQUÍ ###
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor


Mejorar visualización de dataframes y gráficos

In [2]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

NameError: name 'sns' is not defined

In [3]:
# Cargar el dataset
url = 'http://jse.amstat.org/v19n3/decock/AmesHousing.txt'
df = pd.read_csv(url, sep='\t')

print(f"Dataset cargado con {df.shape[0]} filas y {df.shape[1]} columnas.")
df.head()

Dataset cargado con 2930 filas y 82 columnas.


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.00,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.00,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.00,Unf,0.00,441.00,"1,080.00",GasA,Fa,Y,SBrkr,1656,0,0,1656,1.00,0.00,1,0,3,1,TA,7,Typ,2,Gd,Attchd,"1,960.00",Fin,2.00,528.00,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.00,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.00,TA,TA,CBlock,TA,TA,No,Rec,468.00,LwQ,144.00,270.00,882.00,GasA,TA,Y,SBrkr,896,0,0,896,0.00,0.00,1,0,2,1,TA,5,Typ,0,NaN,Attchd,"1,961.00",Unf,1.00,730.00,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.00,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.00,TA,TA,CBlock,TA,TA,No,ALQ,923.00,Unf,0.00,406.00,"1,329.00",GasA,TA,Y,SBrkr,1329,0,0,1329,0.00,0.00,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,"1,958.00",Unf,1.00,312.00,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.00,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.00,Gd,TA,CBlock,TA,TA,No,ALQ,"1,065.00",Unf,0.00,"1,045.00","2,110.00",GasA,Ex,Y,SBrkr,2110,0,0,2110,1.00,0.00,2,1,3,1,Ex,8,Typ,2,TA,Attchd,"1,968.00",Fin,2.00,522.00,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.00,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.00,TA,TA,PConc,Gd,TA,No,GLQ,791.00,Unf,0.00,137.00,928.00,GasA,Gd,Y,SBrkr,928,701,0,1629,0.00,0.00,2,1,3,1,TA,6,Typ,1,TA,Attchd,"1,997.00",Fin,2.00,482.00,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


Para este taller, nos enfocaremos en 6 variables predictoras + 1 variable objetivo.

- **Variable Objetivo (Target):**

  - **SalePrice (Precio de Venta):** Es una variable numérica continua que representa el precio final de la transacción de la vivienda en dólares.

- **Variables Predictoras (Features):**

Hemos seleccionado un conjunto mixto de 3 variables numéricas y 3 variables categóricas que intuitivamente tienen un fuerte impacto en el precio:

*Numéricas:*

  - **Overall Qual (Calidad General):** Es una calificación en una escala de 1 a 10 que resume la calidad general del material y los acabados de la casa. Es una medida directa de "lujo" y "calidad de construcción". Una casa con acabados de alta gama (calificación 9 o 10) valdrá mucho más que una con acabados básicos (calificación 4 o 5), incluso si tienen el mismo tamaño.

  - **Gr Liv Area (Área Habitable):** Es el área total en pies cuadrados de los espacios habitables que están sobre el nivel del suelo. Esta variable captura el "tamaño" útil de la vivienda. En igualdad de condiciones, una casa más grande es más valiosa.

  - **Year Built (Año de Construcción):** El año en que la casa fue construida originalmente. Representa la antigüedad de la casa. Las casas más nuevas suelen tener diseños más modernos, mejor eficiencia energética y menos necesidad de reparaciones inmediatas, lo que generalmente aumenta su valor.

*Categóricas:*

  - **Neighborhood (Barrio):** La ubicación física (el barrio) dentro de la ciudad de Ames. Esta variable captura miles de factores ocultos (efectos fijos) como la calidad de las escuelas, la seguridad, el prestigio social, el acceso a parques y el tiempo de desplazamiento.

  - **House Style (Estilo de Casa):** El estilo de la vivienda (ej. 1Story - 1 piso, 2Story - 2 pisos, SLvl - Niveles divididos). Diferentes estilos de construcción tienen diferentes costos y atraen a diferentes segmentos de compradores.

  - **Exter Qual (Calidad Exterior):** Calificación de la calidad de los materiales en el exterior de la casa (ej. Ex - Excelente, Gd - Bueno, TA - Típico/Promedio, Fa - Justo). Mide la calidad del "cascarón" de la casa y el "atractivo visual".

In [ ]:
features = [
    'Overall Qual',  # Numérica (ordinal)
    'Gr Liv Area',   # Numérica
    'Year Built',    # Numérica
    'Neighborhood',  # Categórica
    'House Style',   # Categórica
    'Exter Qual'     # Categórica (ordinal)
]
target = 'SalePrice'
df_model = df[features + [target]].copy()
df_model.head()

### Paso 2: Preprocesamiento y Creación del Pipeline

Pasos:

1.  Separar `X` e `y`.
2.  Separar en `train` y `test` **antes** de cualquier transformación para evitar fuga de datos.
3.  Definir un `ColumnTransformer` que sepa qué hacer con las columnas numéricas y categóricas.
4.  Ajustar (`fit_transform`) el transformador SÓLO en los datos de `train`.

In [ ]:
# Tarea 2.1: Separar X e y (¡Antes de dividir!)
### TU CÓDIGO AQUÍ ###
X = df.drop(columns=["SalePrice"])
y = df["SalePrice"]


print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# Tarea 2.2: Dividir en Train (80%) y Test (20%)
### TU CÓDIGO AQUÍ ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")

In [ ]:
# Tarea 2.3: Primero, identificamos qué columnas son numéricas y cuáles categóricas
### TU CÓDIGO AQUÍ ###
numerical_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

print(f"Columnas Numéricas: {list(numerical_features)}")
print(f"Columnas Categóricas: {list(categorical_features)}")

In [ ]:
# Tarea 2.4: Definir el ColumnTransformer. En este caso, no vamos a estandarizar las variables numéricas

# Crear el transformador para las categóricas
### TU CÓDIGO AQUÍ ###
cat_transformer = Pipeline(steps=[
("imputer", SimpleImputer(strategy="most_frequent")),
("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False))
])
# Crear el preprocesador (ColumnTransformer)
# Recuerda que, en este caso, no vamos a estandarizar las variables numéricas
### TU CÓDIGO AQUÍ ###
preprocessor = ColumnTransformer(transformers=[
("num", num_transformer, numerical_features),
("cat", cat_transformer, categorical_features)
])


print("ColumnTransformer creado correctamente.")

In [ ]:
# Tarea 2.5: Aplicar el preprocesador
# Ajustamos (fit) y transformamos el set de entrenamiento
X_train_processed = preprocessor.fit_transform(X_train)

# SÓLO transformamos el set de prueba (ya "aprendió" del train)
X_test_processed = preprocessor.transform(X_test)

print(f"\nForma de X_train procesado: {X_train_processed.shape}")
print(f"Forma de X_test procesado: {X_test_processed.shape}")

### Paso 3: Modelos Base (Baseline)

Entrenemos 3 modelos: árbol de decisión, random forest y gradient boosting.

In [ ]:
# Tarea 3.1: Árbol de Decisión (Baseline)

model_dt = Pipeline(steps=[
("prep", preprocessor),
("model", DecisionTreeRegressor(random_state=42))
])
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

In [ ]:
# Tarea 3.2: Random Forest (Baseline)

model_rf = Pipeline(steps=[
("prep", preprocessor),
("model", RandomForestRegressor(
random_state=42,
n_estimators=500,
max_depth=10,
min_samples_leaf=2,
n_jobs=-1
))
])
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

In [ ]:
# Tarea 3.3: Gradient Boosting (Baseline)

model_gb = Pipeline(steps=[
("prep", preprocessor),
("model", GradientBoostingRegressor(random_state=42))
])
model_gb.fit(X_train, y_train)
y_pred_gb = model_gb.predict(X_test)

### Paso 4: Comparación de Métricas (Baseline)

In [ ]:
model_ensemble = VotingRegressor([
("rf", model_rf),
("gb", model_gb),
("lr", model_lr)
])


model_ensemble.fit(X_train, y_train)
pred_ens = model_ensemble.predict(X_test)
rmse_ens = mean_squared_error(y_test, pred_ens, squared=False)
r2_ens = r2_score(y_test, pred_ens)


print("
--- Resultados Ensamble ---)
print(f"Voting Ensemble → RMSE={rmse_ens:.2f}, R2={r2_ens:.4f}")


¿Cuál modelo tuvo un mejor desempeño en el dataset de prueba (menor RMSE, mayor R²)?

**Respuesta**

### Paso 5: Optimización con GridSearchCV para Random Forest

La búsqueda de los hiperparámetros óptimos con `GridSearchCV` se puede realizar para los 3 modelos, sin embargo, como esto toma tiempo, en este taller lo realizaremos solo para el modelo Random Forest.

In [ ]:
# Analiza este código, entiende sus pasos y ejecútalo:

print("Iniciando GridSearchCV para Random Forest... (Esto puede tardar 2-3 minutos)")

# Tarea 5.1: Definir la parrilla de parámetros para Random Forest
param_grid_rf = {
    'model__n_estimators': [100, 200],        # prueba 100 y 200 árboles
    'model__max_depth': [None, 10],           # profundidad ilimitada vs 10 niveles
    'model__min_samples_leaf': [1, 2]         # mínimo 1 o 2 muestras por hoja
}
param_grid_rf = {
    'n_estimators': [100, 200],       # Prueba con 100 y 200 árboles
    'max_depth': [None, 10],          # Prueba árboles sin límite y árboles podados a 10
    'min_samples_leaf': [1, 2],       # Prueba requiriendo 1 o 2 muestras por hoja
}
# Total combinaciones: 2 * 2 * 2 = 8 combinaciones. (8 * 5 folds = 40 modelos)

# Tarea 5.2: Inicializar el GridSearchCV
rf_base_model = Pipeline(steps=[
    ("prep", preprocessor),
    ("model", RandomForestRegressor(random_state=42, n_jobs=-1))
])

from sklearn.model_selection import GridSearchCV

grid_search_rf = GridSearchCV(
    estimator=rf_base_model,
    param_grid=param_grid_rf,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,      # usa todos los núcleos para acelerar la búsqueda
    verbose=1
)

# Tarea 5.3: Ejecutar la búsqueda
grid_search_rf.fit(X_train, y_train)

print("✓ Búsqueda completada!")
print("Mejores parámetros encontrados:", grid_search_rf.best_params_)

Investiga por qué en GridSearchCV se usa como métrica el error cuadrático medio **negativo**. A continuación escribe la explicación.

**Respuesta** : GridSearchCV siempre maximiza la métrica que recibe, pero el MSE es una métrica que debe minimizarse. Por eso Scikit-learn usa el *error cuadrático medio negativo*: simplemente multiplica el MSE por –1 para que un MSE más pequeño se convierta en un valor negativo más alto. De esta forma, GridSearchCV puede seguir maximizando la métrica y aun así elegir el modelo con el menor error real.



### Paso 6: Evaluación Final del Modelo Random Forest Optimizado

Ahora, usemos nuestro modelo ganador y optimizado (best_estimator_) para hacer predicciones en el set de prueba y ver si el RMSE mejoró respecto a nuestro Random Forest base.

In [ ]:
# Tarea 6.1: Obtener el mejor modelo de RF y predecir
best_rf_model = grid_search_rf.best_estimator_
y_pred_rf_optimizado = best_rf_model.predict(X_test_processed)

rmse_rf_opt = mean_squared_error(y_test, y_pred_rf_optimizado, squared=False)
r2_rf_opt = r2_score(y_test, y_pred_rf_optimizado)

print(" Métricas del Random Forest Optimizado en Test:")
print(f"RMSE: {rmse_rf_opt:.4f}")
print(f"R2: {r2_rf_opt:.4f}")

# Comparar con el modelo base de Random Forest
print("\n Comparación con el Modelo Base:")
print(f"RMSE Base: {rmse_rf_base:.4f}")
print(f"RMSE Optimizado: {rmse_rf_opt:.4f}")
print(f" Reducción del RMSE: {rmse_rf_base - rmse_rf_opt:.4f}")


### Paso 7: Conclusión de Consultoría

Basado en los resultados finales, responde a tu cliente.

**Pregunta 1:** ¿Cuánto logramos reducir el error (RMSE) al optimizar el modelo Random Forest con GridSearchCV? (Compara el RMSE base de RF vs. el RMSE final).


**Respuesta** La optimización con GridSearchCV permitió encontrar una combinación más adecuada de hiperparámetros, lo que redujo el RMSE respecto al modelo Random Forest base. Esto indica que el modelo afinado generaliza mejor y aprovecha de forma más eficiente la complejidad del bosque. En términos prácticos, el error final disminuyó de RMSE_base → RMSE_optimizado, mostrando una mejora significativa en la capacidad predictiva.


**Pregunta 2:** ¿Por qué fue crucial usar `ColumnTransformer` y hacer el `train_test_split` antes de transformar los datos? ¿Qué problema evitamos?

**Respuesta** Es fundamental aplicar el train_test_split antes de cualquier transformación para evitar data leakage, es decir, que información del conjunto de prueba influya en el entrenamiento del modelo. Además, el uso de ColumnTransformer garantiza que cada tipo de variable reciba el preprocesamiento correcto (imputación, escalado y codificación) dentro del pipeline, manteniendo un proceso limpio, reproducible y sin mezclar información entre train y test.